In [ ]:
import numpy as np
import sys 
from matplotlib import pyplot as plt
sys.path.insert(1, r"C:\Users\M\Documents\phdmatlab\sqib-pmma-probe-wavelength\UV_Setup\new_parallel_pol_pump653nm")
from ArtrayAnalysis import ArtFit
import plotHelperLatex
plotHelperLatex.setMatplotSettings()

#newest calib
#2024.01.15
CalibNewUnopt = r"C:\Users\M\Documents\phdmatlab\sqib-pmma-probe-wavelength\UV_Setup\new_parallel_pol_pump653nm\ArtrayCalib\CameraData\Artray\2024.01.15\Artray653nmCalib.txt"
CalibNewOptimized = r"C:\Users\M\Documents\phdmatlab\sqib-pmma-probe-wavelength\UV_Setup\new_parallel_pol_pump653nm\ArtrayCalib\CameraData\Artray\2024.01.15\Artray653nmCalibOptimized.txt"
JSONnew = r"C:\Users\M\Documents\phdmatlab\sqib-pmma-probe-wavelength\UV_Setup\new_parallel_pol_pump653nm\JSON_ArtrayBG_2024.01.16.json"

artUnopt = ArtFit(JSONnew, CalibNewUnopt, 1)
artOpt = ArtFit(JSONnew,CalibNewOptimized, 1)
artUnopt.plainFit(1, 3)
artOpt.plainFit(1, 3)

#october not great
CalibOctober = r"C:\Users\M\Documents\phdmatlab\sqib-pmma-probe-wavelength\UV_Setup\new_parallel_pol_pump653nm\ArtrayCalib\CameraData\Artray\2023.10.30\Pump653Calib.txt"
JSONoct = r"C:\Users\M\Documents\phdmatlab\sqib-pmma-probe-wavelength\UV_Setup\new_parallel_pol_pump653nm\JSON_ArtrayBG_2023.10.30.json"

artOct = ArtFit(JSONoct, CalibOctober, 1)
artOct.plainFit(1, 3)

def autoRestrictAndPlot(ax_handle, Artrayhandle, title=""):
    ax_handle.imshow(Artrayhandle.probeMap-Artrayhandle.bgMap, vmin=0)
    #find center and sig
    #x_center, sig,...
    Xfit_parameters = Artrayhandle.XprobeCoeffs[:2]
    Yfit_parameters = Artrayhandle.YprobeCoeffs[:2]
    ax_handle.axis('equal')
    ax_handle.set_ylim((np.ones(2)*(Yfit_parameters[0] +4*Yfit_parameters[1]*np.array([-1,1]))))
    ax_handle.set_xlim((np.ones(2)*(Xfit_parameters[0] +4*Xfit_parameters[1]*np.array([-1,1]))))
    ax_handle.set_ylabel('px / 1')
    ax_handle.set_xlabel('px / 1')
    if title != "":
        ax_handle.set_title(title, loc="left")
    
#fig, ax = plt.subplots(1,3,figsize=plotHelperLatex.figSizer(1,2.8), dpi=288)
#autoRestrictAndPlot(ax[0], artOpt, "a)")
#autoRestrictAndPlot(ax[1], artUnopt, "b)")
#autoRestrictAndPlot(ax[2], artOct, "c)")
#plt.tight_layout()

fig1, ax1 = plt.subplots(1,1,figsize=plotHelperLatex.figSizer(3,1.2), dpi=288)
fig2, ax2 = plt.subplots(1,1,figsize=plotHelperLatex.figSizer(3,1.2), dpi=288)
fig3, ax3 = plt.subplots(1,1,figsize=plotHelperLatex.figSizer(3,1.2), dpi=288)

autoRestrictAndPlot(ax1, artOpt, ".")
autoRestrictAndPlot(ax2, artUnopt)
autoRestrictAndPlot(ax3, artOct)

### Mirror consistency

In [ ]:
import numpy as np
import sys 
sys.path.insert(1, r"C:\Users\M\Documents\phdmatlab\sqib-pmma-probe-wavelength\UV_Setup\new_parallel_pol_pump653nm")
from ArtrayAnalysis import ArtFit
from matplotlib import pyplot as plt
import plotHelperLatex
plotHelperLatex.setMatplotSettings()
#"C:\Users\M\Documents\phdmatlab\sqib-pmma-probe-wavelength\UV_Setup\new_parallel_pol_pump653nm\ArtrayCalib\CameraData\Artray\14.07.2023\MirrorConsistencyTest\01.tif"
directory = r"C:\Users\M\Documents\phdmatlab\sqib-pmma-probe-wavelength\UV_Setup\new_parallel_pol_pump653nm\ArtrayCalib\CameraData\Artray\14.07.2023\MirrorConsistencyTest"+"\\"[0]
fig, ax = plt.subplots(1,1, figsize=plotHelperLatex.figSizer(6,2/3), dpi=288)
filestring = r"{0:02d}.tif"
for ind in range(10):
    map, size = ArtFit.loadTIFF(directory+filestring.format(ind+1))
    x = np.arange(0,size[0], 1)
    y = np.arange(0,size[1], 1)
    xAr, yAr = np.meshgrid(x,y)
    ax.contour(xAr, yAr, map, levels=3)
    
ax.set_xlim((0, size[0]))
ax.set_ylim((0, size[1]))
ax.axis('equal')
ax.set_xlabel('px / 1')
ax.set_ylabel('px / 1')
#plt.show()

#background
#second part
directory = r"C:\Users\M\Documents\phdmatlab\sqib-pmma-probe-wavelength\UV_Setup\new_parallel_pol_pump653nm\ArtrayCalib\CameraData\Artray\14.07.2023\MirrorConsistencyTest"+"\\"[0]
fig2, ax2= plt.subplots(1,1, figsize=plotHelperLatex.figSizer(4,2/3), dpi=288)
filestring = r"{0:02d}.txt"
for ind in range(4):
    map = ArtFit.convertByteToInt(ArtFit.loadByteArr(directory+filestring.format(ind+11)), 200, 300)
    x = np.arange(0,size[0], 1)
    y = np.arange(0,size[1], 1)
    xAr, yAr = np.meshgrid(x,y)
    ax.contour(xAr, yAr, map, levels=3)
    
ax2.set_xlim((0, size[0]))
ax2.set_ylim((0, size[1]))
ax2.axis('equal')
ax2.set_xlabel('px / 1')
ax2.set_ylabel('px / 1')


### Gaussian focusing

In [ ]:
import numpy as np 
import matplotlib.pyplot as plt 
from scipy import constants as const
import plotHelperLatex
plotHelperLatex.setMatplotSettings()
#plt.rcParams['text.usetex'] = True

#Sellmeier equation
def n_squared(lamb, coeffs):
    '''lamb in µm'''
    n2 = 1
    lsquare = lamb**2
    for index in range(3):
        n2 = n2 + coeffs[index, 0]*lsquare/(lsquare-coeffs[index, 1]**2)
    return n2

#Lensmaker equation
def lensmaker(n, R1, R2, d=0):
    '''Returns 1/focal length'''
    f_inv = (n-1)*(1/R1 - 1/R2 + (n-1)*d/(n*R1*R2))
    return f_inv


def wavToEnergy(lamb):
    #Energy in eV, lamb in nm
    return const.h*const.c/lamb*1e9/const.elementary_charge

def energyToWav(E):
    #Energy in eV, lamb in nm
    return const.h*const.c/E*1e9/const.elementary_charge

#https://refractiveindex.info/?shelf=glass&book=fused_silica&page=Malitson
#Maltison 1965 UV fused silica: 0.21 µm - 3.71 µm
coeffsUVFS = np.array([[0.6961663, 0.0684043], [0.4079426, 0.1162414], [0.8974794, 9.896161]])

#Energy range is 1.3 eV to 5 eV in .1 eV steps
energies = np.linspace(1.3, 5, 37)*const.elementary_charge
wavs = const.h*const.c/energies


In [ ]:
#according to edmund
import numpy as np
alpha_calc = lambda f,s, zr_in: f/np.sqrt((abs(s)-f)**2+zr_in**2)
zr  = lambda wav, w0: np.pi*w0**2/wav

def waist_anyplane(w0, f, wav, z, s):
    wL = w0/zr(wav,w0)*np.sqrt(f**2-2*(abs(s)-f)*z + (z/alpha_calc(f,s,zr(wav,w0)))**2)
    return wL
f = 125e-3

s = f + np.linspace(-5e-4, zr(650e-9,3e-3)/2, 200)

print(zr(400e-9,3e-3))

def waist_custom(w0, f, wav, z):
    return w0/zr(wav,w0)*np.sqrt((z/alpha_calc(f,f, zr(wav, w0)))**2+ f**2)


def minimum_waist_s(w0,f,wav,z,s):
    w0/zr(wav,w0)*(2*z/alpha_calc(f,s,zr(wav,w0)**2))*(f**2-2*z*(abs(s)-f)+z**2/alpha_calc(f,s,zr(wav,w0)**2))**(-1/2)
print(waist_custom(3e-3, 125e-3, 653e-9, np.linspace(-5e-4, 1e-2, 20)))
print(waist_anyplane(3e-3, 125e-3, 653e-9, np.linspace(-5e-4, 1e-2, 20), 125e-3))

In [ ]:
from scipy import constants as const 
import numpy as np
from matplotlib import pyplot

#Katz Optimum focusing of Gaussian laser beams: beam waist shift in spot size minimization 1994
#w0 = 3e-3 #m
#lamb = 653e-9 #m
#z_rayleigh = np.pi*w0**2/lamb 
#f_lens = 125e-3
#s = 125e-3
#m = f_lens/np.sqrt((s-f_lens)**2 + z_rayleigh**2)
#w0_min = w0*m
#print(w0_min)
wav = 653e-9
w0_c = lambda lamb, theta: lamb/np.pi/theta
zr_c = lambda lamb, w0: np.pi*w0**2/lamb

n = np.sqrt(n_squared(wav*1e6,coeffsUVFS))
f = 1/lensmaker(n, 59.4, np.inf)*1e-3

z_min = lambda wav, w0, s, f: alpha_calc(f,s,zr(wav,w0))**2*(abs(s)-f)

diameters = np.array([1,2,3,4])*1e-3

fig1, axs1 = plt.subplots(1,1, layout='constrained', figsize = plotHelperLatex.figSizer(2,1.4), dpi=144)
fig2, axs2 = plt.subplots(1,1, layout='constrained', figsize = plotHelperLatex.figSizer(1.6, 1.4), dpi=144)
fig3, axs3 = plt.subplots(1,1, layout='constrained', figsize = plotHelperLatex.figSizer(2,1.4), dpi=144)
#fig.suptitle("653 nm beam waist")

z = np.linspace(-5e-4, 2e-3, 100)
labelz = z[0]
s_in = 40#m
for index in range(len(diameters)):
    axs1.plot(z*1e3, 2e6*waist_anyplane(diameters[index]/2, f, wav, z, s=s_in), label="%.1f mm" %(diameters[index]*1e3))
    z_temp = z_min(wav,diameters[index]/2, s_in, f)

    #axs1.plot(1e3*z_temp, 2e6*waist_anyplane(diameters[index]/2, f, wav, z_temp, s=s_in), marker="+", color = fig1.gca().lines[-1].get_color())
    #axs1.plot(z*1e3, 2e6*waist_custom(diameters[index]/2, f, wav, z), label="%.1f mm" %(diameters[index]*1e3))
    #axs1.text(labelz*1e3-0.65, 2e6*waist_custom(diameters[index]/2, f, wav, z[0]), s="doot")
    #axs1.text(labelz*1e3-0.65, 2e6*waist_anyplane(diameters[index]/2, f, wav, labelz, s=s_in), s="%.1f mm" %(diameters[index]*1e3), fontdict = {"fontsize": "small", "va": "center", "color": fig1.gca().lines[-1].get_color()})
axs1.set_xlabel("z / mm")
axs1.set_ylabel("beam waist / µm")
#axs1.set_xlim((-0.5, 2))
axs1.set_ylim((20,120))
axs1.yaxis.tick_right()
axs1.yaxis.set_label_position('right')
axs1.grid(True)


s_range = np.logspace(-3, 3, base=10, num=40)

axs3.plot(s_range, z_min(wav,1e-3,s_range,f))
axs3.plot(s_range, z_min(wav,2e-3,s_range,f))
axs3.plot(s_range, z_min(wav,3e-3,s_range,f))
axs3.set_xlabel('distance of beam waist to lens')
axs3.set_ylabel('maximum radiance from focal length')
axs3.set_xscale('log')
z = np.linspace(-2e-4, 1e-3, 200)

labelz = z[-1]
for index in range(len(diameters)):
    axs2.plot(z*1e3, (waist_custom(diameters[index]/2, f, wav, 0)/waist_custom(diameters[index]/2, f, wav, z))**2, label="%.1f mm" %(diameters[index]*1e3))
    axs2.text(labelz*1e3+0.01, (waist_custom(diameters[index]/2, f, wav, 0)/waist_custom(diameters[index]/2, f, wav, labelz))**2, s="%.1f mm" %(diameters[index]*1e3), fontdict = {"fontsize": "small", "va": "center", "color": fig2.gca().lines[-1].get_color()})
axs2.set_xlabel("z / mm")
axs2.set_ylabel("relative maximum intensity / a.u.")
axs2.set_xlim((-0.1,1))
axs2.set_yticks([0.5, 0.6, 0.7, 0.8, 0.9, 1.0])
axs2.grid(True)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

import plotHelperLatex
plotHelperLatex.setMatplotSettings()

#taken from edmund optics: Laser Optics and Resource Guide Section 2: Gaussian Beam Propagation

w_c = lambda w0, lamb, z: w0*np.sqrt(1+(lamb*z/(np.pi*w0**2))**2)
w0_c = lambda lamb, theta: lamb/np.pi/theta
zr_c = lambda lamb, w0: np.pi*w0**2/lamb

#collimated beam through lens with focal distance f
#taken from angular aperture
theta_c = lambda D, f: np.arctan(D/(2*f))


dia = 2.5e-3 # m
wavs = np.array([250,300, 400, 500, 600, 700, 800, 900])*1e-9 #wavs in m
n = np.sqrt(n_squared(wav*1e6,coeffsUVFS))
f = 1/lensmaker(n, 59.4, np.inf)*1e-3
thet = theta_c(dia, f)
w0 = w0_c(wavs, thet)

zr = np.max(zr_c(wavs, w0))

z = np.linspace(-2e-4, zr/4, 200)
zlabel = z[-1]
plt.figure()
fig1, axs1 = plt.subplots(1,1, layout='constrained', figsize = plotHelperLatex.figSizer(2,1.4), dpi=144)
#plt.title("Beam waist vs wavelength behavior at 2 mm collimated waist")
for index in range(len(wavs)):
    axs1.plot(z*1e3, w_c(w0[index], wavs[index], z)*2e6, label="%.1f nm" %(wavs[index]*1e9))
    axs1.text(zlabel*1e3+0.01, w_c(w0[index], wavs[index], zlabel)*2e6, s="%.0f nm" %(wavs[index]*1e9), fontdict = {"fontsize": "small", "va": "center", "color": plt.gca().lines[-1].get_color()})
axs1.set_xlabel("z / mm")
axs1.set_ylabel("beam waist / µm")
axs1.set_xlim((-0.2, 1.05))
#plt.legend()



plt.show()